In [3]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.1'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:8 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Reading package lists... Done


In [9]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [10]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [12]:
# 2. Create a temporary view of the DataFrame.

spark.sparkContext.addFile(url)
df=spark.read.csv("file://"+SparkFiles.get("home_sales_revised.csv"),header=True, inferSchema=True)
df.createOrReplaceTempView("home_sales")
df

DataFrame[id: string, date: date, date_built: int, price: int, bedrooms: int, bathrooms: int, sqft_living: int, sqft_lot: int, floors: int, waterfront: int, view: int]

In [17]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
avg_price=spark.sql("select round(avg (price),2), year (date) from home_sales where bedrooms=4 group by year(date)" ).show()



+--------------------+----------+
|round(avg(price), 2)|year(date)|
+--------------------+----------+
|           296363.88|      2022|
|            300263.7|      2019|
|           298353.78|      2020|
|           301819.44|      2021|
+--------------------+----------+



In [18]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?

avg_price_3bdr=spark.sql("select round(avg (price),2), date_built from home_sales where bedrooms=3 and bathrooms=3 group by date_built" ).show()

+--------------------+----------+
|round(avg(price), 2)|date_built|
+--------------------+----------+
|            288770.3|      2015|
|           295962.27|      2013|
|           290852.27|      2014|
|           293683.19|      2012|
|           290555.07|      2016|
|           292859.62|      2010|
|           291117.47|      2011|
|           292676.79|      2017|
+--------------------+----------+



In [19]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

avg_price_2ksf=spark.sql("select round(avg (price),2), date_built from home_sales where bedrooms=3 and bathrooms=3 and floors=2 and sqft_living>=2000 group by date_built" ).show()

+--------------------+----------+
|round(avg(price), 2)|date_built|
+--------------------+----------+
|           297609.97|      2015|
|           303676.79|      2013|
|           298264.72|      2014|
|           307539.97|      2012|
|            293965.1|      2016|
|           285010.22|      2010|
|           276553.81|      2011|
|           280317.58|      2017|
+--------------------+----------+



In [21]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()
avg_price_view_rating=spark.sql("select round(avg (price),2), view from home_sales where price>=350000 group by view" ).show()


print("--- %s seconds ---" % (time.time() - start_time))

+--------------------+----+
|round(avg(price), 2)|view|
+--------------------+----+
|           399856.95|  31|
|          1056336.74|  85|
|           736679.93|  65|
|            755214.8|  53|
|          1080649.37|  78|
|           401419.75|  34|
|          1053472.79|  81|
|           402124.62|  28|
|          1058802.78|  76|
|           401506.97|  26|
|           399537.66|  27|
|           400598.05|  44|
|           401501.32|  12|
|          1137372.73|  91|
|           402022.68|  22|
|          1026006.06|  93|
|            398447.5|  47|
|           401044.25|   1|
|           733780.26|  52|
|           398917.98|  13|
+--------------------+----+
only showing top 20 rows

--- 0.7583422660827637 seconds ---


In [23]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales" ).show()


++
||
++
++



In [24]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [25]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()
avg_price_view_rating=spark.sql("select round(avg (price),2), view from home_sales where price>=350000 group by view" ).show()


print("--- %s seconds ---" % (time.time() - start_time))


+--------------------+----+
|round(avg(price), 2)|view|
+--------------------+----+
|           399856.95|  31|
|          1056336.74|  85|
|           736679.93|  65|
|            755214.8|  53|
|          1080649.37|  78|
|           401419.75|  34|
|          1053472.79|  81|
|           402124.62|  28|
|          1058802.78|  76|
|           401506.97|  26|
|           399537.66|  27|
|           400598.05|  44|
|           401501.32|  12|
|          1137372.73|  91|
|           402022.68|  22|
|          1026006.06|  93|
|            398447.5|  47|
|           401044.25|   1|
|           733780.26|  52|
|           398917.98|  13|
+--------------------+----+
only showing top 20 rows

--- 0.5300452709197998 seconds ---


In [26]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
df.write.partitionBy("date_built").parquet("output")

In [28]:
# 11. Read the parquet formatted data.
parquet_formatted_data=spark.read.parquet("output")

In [29]:
# 12. Create a temporary table for the parquet data.
parquet_formatted_data.createOrReplaceTempView("parquet")
parquet_formatted_data

DataFrame[id: string, date: date, price: int, bedrooms: int, bathrooms: int, sqft_living: int, sqft_lot: int, floors: int, waterfront: int, view: int, date_built: int]

In [30]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version.

start_time = time.time()
avg_price_view_rating_prk=spark.sql("select round(avg (price),2), view from parquet where price>=350000 group by view" ).show()


print("--- %s seconds ---" % (time.time() - start_time))

+--------------------+----+
|round(avg(price), 2)|view|
+--------------------+----+
|           399856.95|  31|
|          1056336.74|  85|
|           736679.93|  65|
|            755214.8|  53|
|          1080649.37|  78|
|           401419.75|  34|
|          1053472.79|  81|
|           402124.62|  28|
|          1058802.78|  76|
|           401506.97|  26|
|           399537.66|  27|
|           400598.05|  44|
|           401501.32|  12|
|          1137372.73|  91|
|           402022.68|  22|
|          1026006.06|  93|
|            398447.5|  47|
|           401044.25|   1|
|           733780.26|  52|
|           398917.98|  13|
+--------------------+----+
only showing top 20 rows

--- 1.0939080715179443 seconds ---


In [31]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales" ).show()

++
||
++
++



In [33]:
# 15. Check if the home_sales is no longer cached

spark.catalog.isCached('home_sales')

False